# IE 7275_HW3_Lingxuan_Ye

## Imports

In [ ]:
from typing import Literal

import numpy as np
import pandas as pd
import sympy
import scikitplot as skplt

from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score

## Problem 1

### a

In [ ]:
data = pd.read_csv("banks-1.csv")
data

In [ ]:

lreg = LogisticRegression(solver="liblinear")

## 3.8 and above ##

# lreg.fit(
#     x:=data[["TotExp/Assets", "TotLns&Lses/Assets"]],
#     data["Financial Condition"]
# )

x = data[["TotExp/Assets", "TotLns&Lses/Assets"]]
lreg.fit(x, data["Financial Condition"])

data["Predict"] = lreg.predict(x)
data

### b

In [ ]:
lreg.intercept_

In [ ]:
lreg.coef_

$$
\begin{align*}
{Logit}&=0.130104\times{TotExp/Assets}+0.55673992\times{TotLns\&Lses/Assets}-0.30280261\\
{Odds}&=e^{0.130104}\times{TotExp/Assets}+0.55673992\times{TotLns\&Lses/Assets}-0.30280261\\
{Probability}&=\frac{1}{e^{-(0.130104\times{TotExp/Assets}+0.55673992\times{TotLns\&Lses/Assets}-0.30280261)}+1}\\
\end{align*}
$$

### c

In [ ]:
logit = lreg.coef_[0, 0] * 0.11 + lreg.coef_[0, 1] * 0.6 - lreg.intercept_[0]
odds = np.exp(logit)
prob = 1 / (1 + np.exp(- logit))
print(
    f"Logit = {logit}\n"
    f"Odds = {odds}\n"
    f"Probability = {prob}"
)

Classified as being financially weak.

### d

In [ ]:
logit, prob = sympy.symbols("logit, prob")
sympy.solve(
    prob - 1 / (sympy.E ** (-logit) + 1),
    logit
)[0].subs(prob, 0.5)

Treshhold for logit is 0.

In [ ]:
(sympy.E ** logit).subs(logit, 0)

Treshhold for odds is 1.

### e

In [ ]:
np.exp(lreg.coef_[0, 1])

The odds of being financially weak will increase by 1.74 times as total loans & leases increases 1 unit.

### f

Decrease.

## Problem 2

### a

$$
\begin{align*}
{Logit(Y=1)}&=1.13\times{Experience}+0.18\times{Training}-10.98\\
{Probability(Y=1)}&=\frac{1}{e^{-(1.13\times{Experience}+0.18\times{Training}-10.98)}+1}\\
\end{align*}
$$

### b

In [ ]:
np.exp(1.13)


The probability of finishing task on time will increase by 3.10 times as experience increases 1 unit.

### c

In [ ]:
args = np.array([[9, 8], [9, 7], [8, 7]])
probs = 1 / (np.exp(-(0.18 * args[:, 1] + 1.13 * args[:, 0] - 10.98)) + 1)
probs

## Problem 3

In [ ]:
data = pd.read_excel("Universalbank_1500.xlsx")

family_dummy = pd.get_dummies(data["Family"])
family_dummy.columns = (f"Family={i+1}" for i in range(family_dummy.shape[1]))
edu_dummy = pd.get_dummies(data["Education"])
edu_dummy.columns = (f"Education={i+1}" for i in range(edu_dummy.shape[1]))

data = pd.concat((data, family_dummy, edu_dummy), axis=1)
data

In [ ]:
cls = data["Personal Loan"]
to_drop = ["ID", "ZIP Code", "Personal Loan", "Family", "Education"]
data_ = data.drop(to_drop, axis=1)
train = data_.sample(frac=0.6)
train_cls = cls[cls.index.isin(train.index)]
valid = data_.drop(train.index)
valid_cls = cls.drop(train.index)

valid_, valid_cls_ = valid.copy(), valid_cls.copy()

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(train, train_cls)
predicted = lda.predict(valid)
prob = lda.predict_proba(valid)
prob_0, prob_1 = prob[:, 0], prob[:, 1]
prob_0

### a

In [ ]:
def desc(type_: Literal["acceptors", "nonacceptors"]) -> None:
    assert type_.lower() in ("acceptors", "nonacceptors")
    bin_ = 1 if type_ == "acceptors" else 0
    num_cols = ["Age", "Experience", "Income", "CCAvg", "Mortgage"]
    temp = data[data["Personal Loan"] == bin_].drop(to_drop, axis=1)
    num = temp[num_cols]
    cat = temp.drop(num_cols, axis=1)
    yield num.describe()
    yield pd.DataFrame(cat.sum() / cat.shape[0],columns=["Percentage"])

#### Acceptors

In [ ]:
desc_ = desc("acceptors")

##### Numerical

In [ ]:
next(desc_)

##### Categorical

In [ ]:
next(desc_)

#### Noncceptors

In [ ]:
desc_ = desc("nonacceptors")

##### Numerical

In [ ]:
next(desc_)

##### Categorical

In [ ]:
next(desc_)

Substantially different predictors:

- `percentage of CD account`

- `Education=1`

- `mean Income`

- `mean Morgage`

- `mean CCAvg`

### b

In [ ]:
matrix = confusion_matrix(valid_cls, predicted)
ConfusionMatrixDisplay(matrix, display_labels=[False, True]).plot()

In [ ]:
# number of cases that predicted True are always 0 for unknown reason
# environment are as follow:

!pip freeze > requirements.txt

from sys import version_info
version_info

#### Accuracy Rate

In [ ]:
matrix[0, 0] / 600

#### Type I

In [ ]:
matrix[1, 0]

#### Type II

In [ ]:
matrix[0, 1]

False negative is more than false positive.

In [ ]:
valid["Actual"] = valid_cls
valid["Predicted"] = predicted
valid["Prob_0"] = prob_0
valid

#### False Positive

In [ ]:
valid[(valid["Actual"] == 0) & (valid["Predicted"] == 1)]

No one is missclassified as acceptor.

#### False Negative

In [ ]:
valid[(valid["Actual"] == 1) & (valid["Predicted"] == 0)].sample(3)

They have relatively high income and families, and are aged and well-educated. Those could be the reason that they are misclassified.

### c

In [ ]:
skplt.metrics.plot_cumulative_gain(valid_cls, prob)

Model trained prefers classifing acceptor instead of nonacceptor.

In [ ]:
decile = pd.DataFrame(
    {
        "Actual": valid_cls,
        "Prob": prob_1
    }
).sort_values("Prob", ascending=False).reset_index()

decile["Gain"] = decile["Actual"].cumsum()

loc = np.random.choice(decile.index, 10)
loc.sort()
percent = decile.loc[loc, "Gain"]
rand = np.array([(i + 1) / 10 for i in range(10)]) * decile.shape[0] / 2
actual_rand_ratio = percent / rand
actual_rand_ratio

#### Decilewise lift chart

In [ ]:
plt.bar(
    np.array([(i + 1) / 10 for i in range(10)]),
    actual_rand_ratio,
    width=0.2
)

### d

In [ ]:
lreg = LogisticRegression(solver="liblinear")
lreg.fit(train, train_cls)
predicted = lreg.predict_proba(valid_)
skplt.metrics.plot_cumulative_gain(valid_cls_, predicted)

New model trained prefers to predict acceptor as well and we cannot determine which model is better from the results.

In [ ]:
decile = pd.DataFrame(
    {
        "Actual": valid_cls_,
        "Prob": predicted[:, 1]
    }
).sort_values("Prob", ascending=False).reset_index()

decile["Gain"] = decile["Actual"].cumsum()

loc = np.random.choice(decile.index, 10)
loc.sort()
percent = decile.loc[loc, "Gain"]
rand = np.array([(i + 1) / 10 for i in range(10)]) * decile.shape[0] / 2
actual_rand_ratio = percent / rand

plt.bar(
    np.array([(i + 1) / 10 for i in range(10)]),
    actual_rand_ratio,
    width=0.2
)

### f

Lower the cut-off value because the cost of false positive is much lower than false negative.

## Problem 5

In [ ]:
data = df = pd.DataFrame(
    [[3.5, 4.0, 1], [2.0, 4.0, 1], [2.0, 4.0, 1], [1.5, 7.0, 1], [7.0, 6.5, 1],
     [2.1, 2.5, 0], [8.0, 4.0, 0], [9.1, 4.5, 0]],
    columns=["A", "B", "Class"])

### a

In [ ]:
x_train=data.drop(["Class"],axis=1)
y_train=data["Class"]

svc = SVC(kernel="linear", C=1)

svc.fit(x_train, y_train)
svc.support_vectors_

In [ ]:
svc.coef_

### b

In [ ]:
svc.intercept_

In [ ]:
svc.dual_coef_

### d

In [ ]:
plt.scatter(x_train.iloc[:, 0], x_train.iloc[:, 1], c=y_train, cmap="winter")
ax = plt.gca()
xlim = ax.get_xlim()
w = svc.coef_[0]
a = -w[0] / w[1]
x = np.linspace(xlim[0], xlim[1])
y = a * x - svc.intercept_[0] / w[1]
plt.plot(x, y)
y = a * x - (svc.intercept_[0] - 1) / w[1]
plt.plot(x, y, "k--")
y = a * x - (svc.intercept_[0] + 1) / w[1]
plt.plot(x, y, "k--")

## Problem 6

In [ ]:
data = pd.read_excel(
    "Universalbank_1500.xlsx").drop(["ZIP Code", "ID"],
    axis=1
)

### a

In [ ]:
b_train = data.sample(frac=2 / 3)
b_valid = data.drop(b_train.index)
x = b_train.drop(["Personal Loan"], axis=1)
y = b_train["Personal Loan"]
x_val = b_valid.drop(["Personal Loan"], axis=1)
y_val = b_valid["Personal Loan"]
svc = SVC(kernel="linear", C=1)
svc.fit(x, y)

### b

In [ ]:
svc.support_vectors_

### c

In [ ]:
shuffled=data.sample(frac=1)
folds=np.array_split(shuffled,5)
fold1=folds[0]

x1=fold1.drop(["Personal Loan"],axis=1)
y1=fold1["Personal Loan"]

fold2=folds[1]
x2=fold2.drop(["Personal Loan"],axis=1)
y2=fold2["Personal Loan"]

fold3=folds[2]
x3=fold3.drop(["Personal Loan"],axis=1)
y3=fold3["Personal Loan"]

fold4=folds[3]
x4=fold4.drop(["Personal Loan"],axis=1)
y4=fold4["Personal Loan"]

fold5=folds[4]
x5=fold5.drop(["Personal Loan"],axis=1)
y5=fold5["Personal Loan"]

In [ ]:
# fold 1

x = pd.concat([x2, x3, x4, x5], axis=0)
y = pd.concat([y2, y3, y4, y5], axis=0)

svc = SVC(kernel="linear", C=1.0)
svc.fit(x, y)
accuracy_score(y1, svc.predict(x1))

In [ ]:
# fold 2

x = pd.concat([x1, x3, x4, x5], axis=0)
y = pd.concat([y1, y3, y4, y5], axis=0)

svc = SVC(kernel="linear", C=1.0)
svc.fit(x, y)
accuracy_score(y2, svc.predict(x2))

In [ ]:
# fold 3

x = pd.concat([x2, x1, x4, x5], axis=0)
y = pd.concat([y2, y1, y4, y5], axis=0)

svc = SVC(kernel="linear", C=1.0)
svc.fit(x, y)
accuracy_score(y3, svc.predict(x3))

In [ ]:
# fold 4

x = pd.concat([x2, x3, x1, x5], axis=0)
y = pd.concat([y2, y3, y1, y5], axis=0)

svc = SVC(kernel="linear", C=1.0)
svc.fit(x, y)
accuracy_score(y4, svc.predict(x4))

In [ ]:
# fold 5

x = pd.concat([x2, x3, x4, x1], axis=0)
y = pd.concat([y2, y3, y4, y1], axis=0)

svc = SVC(kernel="linear", C=1.0)
svc.fit(x, y)
accuracy_score(y5, svc.predict(x5))